<h1 align="center">Coursera Capstone - Final Project</h1>
<p align="center">This project has 3 main data from Chicago City: </p>
<ul>
    <li>Crime Data: <a href="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2">https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2 </a></li>
    <li>Census Data: <a href="https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2">https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2 </a></li>
    <li>Coordinates: <a href="https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6">https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6</a></li>
</ul>

<p>The main goal is taking the last 1000 crimes from Chicago and mapping all those crimes in community areas. Then, after finding latitude and longitude of community areas, insert some surrounding venues from Foursquare. With that data we want to understand how the characteristics and points of community areas influence in criminality levels.</p>

<h3>Packages</h3>

In [1]:
import json
import ast
import folium
import requests
import geocoder
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import statsmodels.api as sm
import matplotlib.colors as colors


from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from shapely.geometry import Polygon

<h3>Crime Data</h3>

In [2]:
crimes = pd.read_csv('Crime.csv')
crimes = crimes[['ID','Primary Type','Community Area']]
crimes['count'] = 1
crimes.head()

,ID,Primary Type,Community Area,count
0,9998220,THEFT,27,1
1,9998221,BATTERY,26,1
2,9998223,BATTERY,32,1
3,9998224,BATTERY,25,1
4,9998225,OTHER OFFENSE,7,1


In [3]:
crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              1000 non-null   int64 
 1   Primary Type    1000 non-null   object
 2   Community Area  1000 non-null   int64 
 3   count           1000 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 31.4+ KB


In [4]:
crimes = crimes.groupby('Community Area').sum()
crimes = crimes.drop('ID',axis=1).reset_index()
crimes.columns = ['Community Area Number', 'Number of crimes']
crimes.head()

,Community Area Number,Number of crimes
0,1,18
1,2,9
2,3,15
3,4,9
4,5,6


<h3>Census Data</h3>

In [5]:
census = pd.read_csv("Census.csv")
census.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6


In [6]:
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 9 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Community Area Number                         77 non-null     int64  
 1   COMMUNITY AREA NAME                           77 non-null     object 
 2   PERCENT OF HOUSING CROWDED                    77 non-null     float64
 3   PERCENT HOUSEHOLDS BELOW POVERTY              77 non-null     float64
 4   PERCENT AGED 16+ UNEMPLOYED                   77 non-null     float64
 5   PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA  77 non-null     float64
 6   PERCENT AGED UNDER 18 OR OVER 64              77 non-null     float64
 7   PER CAPITA INCOME                             77 non-null     int64  
 8   HARDSHIP INDEX                                77 non-null     int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 5.5+ KB


<h3>Census: Latitude and Longitude</h3>

In [7]:
gps = pd.read_csv("community.txt",sep="\t")
gps = gps.groupby('Community Name').mean().reset_index()
gps.columns = ['COMMUNITY AREA NAME', 'Community Area Number', 'Latitude', 'Longitude']
gps.head()

,COMMUNITY AREA NAME,Community Area Number,Latitude,Longitude
0,Albany Park,14,41.966615,-87.719082
1,Archer Heights,57,41.803095,-87.725199
2,Armour Square,34,41.844593,-87.633735
3,Ashburn,70,41.745958,-87.706780
4,Auburn Gresham,71,41.747317,-87.657136


In [8]:
census = census.merge(gps, left_on='Community Area Number',right_on='Community Area Number').drop('COMMUNITY AREA NAME_y',axis=1)
census.head()

,Community Area Number,COMMUNITY AREA NAME_x,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Latitude,Longitude
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39,42.009120,-87.668648
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46,41.999316,-87.692394
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20,41.966222,-87.658792
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17,41.968844,-87.685397
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6,41.950503,-87.681029


<h3>All Together</h3>

In [9]:
census = census.merge(crimes, left_on='Community Area Number',right_on='Community Area Number')
census.head()

,Community Area Number,COMMUNITY AREA NAME_x,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Latitude,Longitude,Number of crimes
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39,42.009120,-87.668648,18
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46,41.999316,-87.692394,9
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20,41.966222,-87.658792,15
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17,41.968844,-87.685397,9
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6,41.950503,-87.681029,6


<h3>Foursquare</h3>

In [10]:
# Chicago latitude and longitude
tor_lat = 41.8339037
tor_lng = -87.8720471

map_chicago = folium.Map(location=[tor_lat, tor_lng], zoom_start=10)

for lat, lng, areaName, crimesN in zip(census['Latitude'], census['Longitude'], census['COMMUNITY AREA NAME_x'], census['Number of crimes']):
    label = '{}, {}'.format(crimesN, areaName)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

In [11]:
# Gets the name of the category
def get_category_type(row):
    try:
        categories_list = row['Category']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h3>FourSquare API on all neighbourhoods</h3>

In [13]:
CLIENT_ID = 'HI3JK0LQVPOUTD1CSM4ZDRMRLUVQ02T53XMOB3U0DXABTGQF' # your Foursquare ID
CLIENT_SECRET = 'H0GGPJOOPFW5FLHG1ESRPWFW2FR05K5SYR0JAOMYWHGJV1GJ' # your Foursquare Secret
VERSION = '20203007'

In [14]:
explore_df_list = []

for i, nbd_name in enumerate(census['COMMUNITY AREA NAME_x']):
        
    try :         
        ### Getting the data of neighbourhood
        nbd_name = census.loc[i, 'COMMUNITY AREA NAME_x']
        nbd_lat = census.loc[i, 'Latitude']
        nbd_lng = census.loc[i, 'Longitude']

        radius = 400 # Setting the radius as 500 metres
        LIMIT = 100 # Getting the top 100 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']
        
        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)

        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            explore_df_list.append([nbd_name, nbd_lat, nbd_lng] + nearby.loc[i, :].values.tolist())
    
    except Exception as e:
        pass

<ipython-input-14-a6620211377f>:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby = json_normalize(results) # Flattens JSON


In [15]:
explore_df = pd.DataFrame([item for item in explore_df_list])
explore_df.columns = ['CA', 'Latitude', 'Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
explore_df.head()

,CA,Latitude,Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,Rogers Park,42.00912,-87.668648,Morse Fresh Market,Grocery Store,42.008087,-87.667041
1,Rogers Park,42.00912,-87.668648,Glenwood Sunday Market,Farmers Market,42.008525,-87.666251
2,Rogers Park,42.00912,-87.668648,The Common Cup,Coffee Shop,42.007797,-87.667901
3,Rogers Park,42.00912,-87.668648,The Glenwood,Bar,42.008502,-87.666273
4,Rogers Park,42.00912,-87.668648,Smack Dab,Bakery,42.009291,-87.666201


In [16]:
# Preprocessing
table_onehot = pd.get_dummies(explore_df[['Venue Category']], prefix="", prefix_sep="")
table_onehot['CA'] = explore_df['CA'] 
fixed_columns = [table_onehot.columns[-1]] + list(table_onehot.columns[:-1])
table_onehot = table_onehot[fixed_columns]
table_onehot = table_onehot.groupby('CA').mean().reset_index()

table = table_onehot

# top 5 venues
num_top_venues = 15
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['CA']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
table_venues= pd.DataFrame(columns=columns)
table_venues['CA'] = census['COMMUNITY AREA NAME_x']

for ind in np.arange(table.shape[0]):
    table_venues.iloc[ind, 1:] = return_most_common_venues(table.iloc[ind, :], num_top_venues)

table_venues.head()

,CA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Rogers Park,Mexican Restaurant,Accessories Store,Bus Station,Bakery,Grocery Store,BBQ Joint,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Park,Chinese Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Eye Doctor
1,West Ridge,Pizza Place,Discount Store,Mexican Restaurant,Seafood Restaurant,Eastern European Restaurant,Sandwich Place,Bank,Bakery,Rental Car Location,Nightclub,Hotel,Design Studio,Cycle Studio,Farmers Market,Falafel Restaurant
2,Uptown,Chinese Restaurant,Park,Pizza Place,Seafood Restaurant,Mexican Restaurant,Storage Facility,Grocery Store,Dim Sum Restaurant,Mobile Phone Shop,Asian Restaurant,English Restaurant,Electronics Store,Yoga Studio,Donut Shop,Ethiopian Restaurant
3,Lincoln Square,Pizza Place,Liquor Store,Bar,Automotive Shop,Yoga Studio,Eastern European Restaurant,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Eye Doctor,Ethiopian Restaurant,English Restaurant
4,North Center,BBQ Joint,Seafood Restaurant,Pharmacy,Dim Sum Restaurant,Video Store,Ethiopian Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Yoga Studio,Eye Doctor,Donut Shop,Farmers Market,Fast Food Restaurant,Field


In [17]:
table_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Columns: 234 entries, CA to Yoga Studio
dtypes: float64(233), object(1)
memory usage: 135.4+ KB


<h3>K-Means</h3>

In [18]:
# Set number of clusters
kclusters = 5
clustering = table_onehot.drop('CA', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(clustering)

aux = pd.DataFrame(data={'CA':table_onehot['CA'],'Clusters':kmeans.labels_})
table_venues = table_venues.merge(aux, left_on='CA',right_on='CA')
table_venues.head()

,CA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,Clusters
0,Rogers Park,Mexican Restaurant,Accessories Store,Bus Station,Bakery,Grocery Store,BBQ Joint,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Park,Chinese Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Eye Doctor,3
1,West Ridge,Pizza Place,Discount Store,Mexican Restaurant,Seafood Restaurant,Eastern European Restaurant,Sandwich Place,Bank,Bakery,Rental Car Location,Nightclub,Hotel,Design Studio,Cycle Studio,Farmers Market,Falafel Restaurant,3
2,Uptown,Chinese Restaurant,Park,Pizza Place,Seafood Restaurant,Mexican Restaurant,Storage Facility,Grocery Store,Dim Sum Restaurant,Mobile Phone Shop,Asian Restaurant,English Restaurant,Electronics Store,Yoga Studio,Donut Shop,Ethiopian Restaurant,3
3,Lincoln Square,Pizza Place,Liquor Store,Bar,Automotive Shop,Yoga Studio,Eastern European Restaurant,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Eye Doctor,Ethiopian Restaurant,English Restaurant,3
4,North Center,BBQ Joint,Seafood Restaurant,Pharmacy,Dim Sum Restaurant,Video Store,Ethiopian Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Yoga Studio,Eye Doctor,Donut Shop,Farmers Market,Fast Food Restaurant,Field,3


In [19]:
census = census.rename(columns={"COMMUNITY AREA NAME_x": "CA"})
table_all = census
table_all = table_all.join(table_venues.set_index('CA'), on='CA')
table_all.dropna(inplace = True)
table_all['Clusters'] = table_all['Clusters'].astype(int)
table_all.head()

,Community Area Number,CA,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Latitude,...,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,Clusters
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39,42.009120,...,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Park,Chinese Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Eye Doctor,3
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46,41.999316,...,Bank,Bakery,Rental Car Location,Nightclub,Hotel,Design Studio,Cycle Studio,Farmers Market,Falafel Restaurant,3
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20,41.966222,...,Grocery Store,Dim Sum Restaurant,Mobile Phone Shop,Asian Restaurant,English Restaurant,Electronics Store,Yoga Studio,Donut Shop,Ethiopian Restaurant,3
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17,41.968844,...,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Eye Doctor,Ethiopian Restaurant,English Restaurant,3
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6,41.950503,...,Eastern European Restaurant,Electronics Store,English Restaurant,Yoga Studio,Eye Doctor,Donut Shop,Farmers Market,Fast Food Restaurant,Field,3


<h3>Clusters in map</h3>

In [20]:
map_clusters = folium.Map(location=[tor_lat, tor_lng], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(table_all['Latitude'], table_all['Longitude'], table_all['CA'], table_all['Clusters']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster) + ')', parse_html=True)
    map_clusters.add_child(
        folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7))
       
map_clusters

<h3>Study Case</h3>

In [21]:
table_all.columns

Index(['Community Area Number', 'CA', 'PERCENT OF HOUSING CROWDED',
       'PERCENT HOUSEHOLDS BELOW POVERTY', 'PERCENT AGED 16+ UNEMPLOYED',
       'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       'PERCENT AGED UNDER 18 OR OVER 64', 'PER CAPITA INCOME ',
       'HARDSHIP INDEX', 'Latitude', 'Longitude', 'Number of crimes',
       '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue',
       '11th Most Common Venue', '12th Most Common Venue',
       '13th Most Common Venue', '14th Most Common Venue',
       '15th Most Common Venue', 'Clusters'],
      dtype='object')

In [22]:
X = table_all[['HARDSHIP INDEX','Clusters']]

y = table_all['Number of crimes']

In [23]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:       Number of crimes   R-squared (uncentered):                   0.566
Model:                            OLS   Adj. R-squared (uncentered):              0.554
Method:                 Least Squares   F-statistic:                              45.73
Date:                Thu, 30 Jul 2020   Prob (F-statistic):                    1.97e-13
Time:                        15:39:34   Log-Likelihood:                         -278.41
No. Observations:                  72   AIC:                                      560.8
Df Residuals:                      70   BIC:                                      565.4
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------